In [26]:
class Tree:
    def fit(self,x_train,y_train,feature_names):
        '''
        x_train should be 2-dimension array
        y_train and feature_names should be 1-dimension array
        
        '''
        def new_log(n,bottom):
            '''
            caculate log value with bottom,when n=0,log value=0.
        
            '''   
            import math as m
            if n==0:
                return 0
            else:
                return m.log(n,bottom)
        
        def LabelEncoder(x):
            '''
            encode array that contains str.
        
            '''
            import numpy as np 
            n_values=x.shape[0]
            new_x=np.full(x.shape[0],np.nan)
            catagories=np.unique(x)
            new_catagories=np.arange(catagories.shape[0])
            for i in range(n_values):
                for j in range(catagories.shape[0]):
                    if x[i]==catagories[j]:
                        new_x[i]=new_catagories[j]
            return new_x.astype(int)
            
        
        def labelencoder(x,border):
            '''
            encode continuous array into 2 catagories:x>border and x<=border.
        
            '''
            n_values=x.shape[0]
            new_x=np.full(x.shape[0],np.nan)
            for k in range(n_values):
                if x[k]>border:
                    new_x[k]=1
                elif x[k]<=border:
                    new_x[k]=0 
            return new_x.astype(int)
        
        def entropy(y):
            '''
            Caculate the entropy.  
            y should be an 1-dimension array as target.
            
            '''
            import numpy as np
            catagories,counts=np.unique(y,return_counts=True)
            sample_size=y.shape[0]
            n_catagories=catagories.shape[0]
            entropy=0
            for i in range(n_catagories):
                entropy-=(counts[i]/sample_size)*new_log(counts[i]/sample_size,2)
            return entropy
        
        def info_gain(x,y):
            '''
            caculate the information gain.
            x should be an 1-dimension array as a column of feature.
            y should be an 1-dimension array as target.
            
            '''
            import numpy as np
            x_catagories,x_counts=np.unique(x,return_counts=True)
            sample_size=x.shape[0]
            n_x_catagories=x_catagories.shape[0]
            weights=np.full(n_x_catagories,np.nan)
            catagory_entropies=np.full(n_x_catagories,np.nan)
            for i in range(n_x_catagories):
                weights[i]=x_counts[i]/sample_size
                catagory_entropies[i]=entropy(y[x==x_catagories[i]]) 
            info_gain= entropy(y)-np.sum(weights*catagory_entropies) 
            return info_gain
        
        def continuous_treatment(x,y,continuous_benchmark=5):
            '''
            discretize an array that is thought to be continuous according to the numbers of value types it has.
            The output is the discretized array and the border that split the continious array calculated by max entropy.
            
            x,y should be 1-dimension array.
            
            '''
            import numpy as np
            n_type_values=np.unique(x).shape[0]
            if n_type_values>=continuous_benchmark:
                n_values=x.shape[0]
                midpoint=np.full(n_values-1,np.nan)
                discrete_x=np.full((n_values-1,n_values),np.nan)
                info_gains=np.full(n_values-1,np.nan)
                value_sort=np.sort(x)
                for i in range(n_values-1):
                    midpoint[i]=(x[i+1]-x[i])/2+x[i]
                    discrete_x[i]=labelencoder(x,midpoint[i])
                    info_gains[i]=info_gain(discrete_x[i],y)
                final_border=np.random.RandomState(123).choice(midpoint[info_gains==np.max(info_gains)])
                return final_border,labelencoder(x,final_border)
            else:
                return 'Originally discrete, no need to discretize',x
        
        def best_feature(x_y,feature_names):
            '''
            Return the feature with max info_gain so that the dataframe can be splited next
            x_y should be an 2-dimension array as features and target.
        
            '''
            import numpy as np
            x,y=np.hsplit(x_y,[x_y.shape[1]-1])
            info_gains=np.full(x.shape[1],np.nan)
            for i in range(x.shape[1]):
                info_gains[i]=info_gain(x[:,i],y)
            best_feature_index=np.random.RandomState(123).choice(np.arange(x.shape[1])[info_gains==np.max(info_gains)])
            best_feature=feature_names[best_feature_index]
            return best_feature,best_feature_index

        def data_split(x_y,best_feature_index,catagory):
            '''
            split the dataframe according to the best_feature_indx,and the catagory of best_feature
            
            '''
            import numpy as np
            sorted_x_y=x_y[x_y[:,best_feature_index].argsort()]
            splited_data=sorted_x_y[sorted_x_y[:,best_feature_index]==catagory]
            return splited_data
            
        def create_tree(x_y,feature_names):
            '''
            create the tree in the form of dictionary

            '''
            import numpy as np
            x,y=np.hsplit(x_y,[x_y.shape[1]-1])
            if np.unique(y).shape[0]==1:
                return str(int(np.unique(y)))
            else:
                best_feat,best_feat_index=best_feature(x_y,feature_names)
                tree={best_feat:{}}
                for catagory in range(np.unique(x[:,best_feat_index]).shape[0]):
                    tree[best_feat][catagory]=create_tree(data_split(x_y,best_feat_index,catagory),feature_names)
                return tree
  
#  Encode data and treat continous data
        import numpy as np
        y_train=LabelEncoder(y_train)
        for i in range(x_train.shape[1]):
            x_train[:,i]=continuous_treatment(x_train[:,i],y_train)[1]
            x_train[:,i]=LabelEncoder(x_train[:,i])
        x_y=np.concatenate((x_train,y_train[:,np.newaxis]),axis=1) 
        tree=create_tree(x_y,feature_names)
        self.tree=tree 
        
    def predict(self,x_test):
        '''
        x_test should be a 2-dimension array

        '''
        def single_predict(tree,single_x_test):
            '''
            this function predict a test point, x_test should be a dictionary of a single data point
            
            '''
            first=list(tree.keys())[0]
            second=single_x_test[first]
            edege=tree[first][second]
            if isinstance(edege,str)==True:
                predict=edege
            else:
                predict=single_predict(edege,single_x_test)
            return predict
        import numpy as np
        empty_x_test=np.full(x_test.shape[0],np.nan)
        for i,single_x_test in enumerate(x_test):
            single_x_test=single_x_test.tolist()
            single_x_test={feature_names:single_x_test for feature_names,single_x_test in zip(feature_names,single_x_test)}
            empty_x_test[i]=single_predict(self.tree,single_x_test) 
        return empty_x_test
        
#test
import numpy as np
x_train=np.array([[0,2,0,0],[0,2,0,1],[1,2,0,0],[2,1,0,0],[2,0,1,0],[2,0,1,1],[1,0,1,1],[0,1,0,0],[0,0,1,0],[2,1,1,0],[0,1,1,1],
            [1,1,0,1],[1,2,1,0],[2,1,0,1]])
y_train=np.array([0,0,1,1,1,0,1,0,1,1,1,1,1,0])
x_test=np.array([[0,2,0,0],[0,2,0,1],[1,2,0,0],[2,1,0,0],[2,0,1,0],[2,0,1,1]])
feature_names=np.array(['age','income','student','credit'])
clf=Tree()
clf.fit(x_train,y_train,feature_names)
clf.tree,clf.predict(x_test)

C:\Users\wyy\AppData\Local\Temp\ipykernel_3804\712809077.py:142: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return str(int(np.unique(y)))


({'age': {0: {'student': {0: '0', 1: '1'}},
   1: '1',
   2: {'credit': {0: '1', 1: '0'}}}},
 array([0., 0., 1., 1., 1., 0.]))